In [1]:
import fireworks
from fireworks.nbodylib import dynamics as dyn
from fireworks.nbodylib import integrators as intg
from fireworks import ic
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

In [4]:
def evolve(particles,integrator,tstep = .001):
        

    period = 10
    tot_time = 0 

    acc_list = []
    potential_list = []
    positions = []
    velocities = []
    energy = []
    jerk_ = []

    init_energy = particles.Etot()

    while tot_time < 10*period:
        tot_time +=tstep

        # Evolve the binary 

        (particles, tstep, acc, jerk, potential) = integrator(particles=particles, 
                                                                            tstep=tstep,
                                                                            acceleration_estimator=dyn.acceleration_direct_vectorized,
                                                                            softening=0.)
        
        acc_list.append(acc)
        potential_list.append(potential)
        positions.append(particles.pos)
        velocities.append(particles.vel)
        energy.append(particles.Etot())
        jerk_.append(jerk)
    
    
    
    potential_list = np.array(potential_list)
    positions = np.array(positions).reshape(len(positions),2,3)
    velocities = np.array(velocities).reshape(len(velocities),2,3)
    energy = np.array(energy)
    return (acc_list, potential_list, positions, velocities, energy,init_energy,jerk_)

In [5]:
particles = ic.ic_two_body(mass1=1,mass2=1,rp=1,e=0)
(acc_list, potential_list, positions, velocities, energy,init_energy,jerk) = evolve(particles,intg.integrator_tsunami)


/root/miniconda3/lib/python3.10/site-packages/fireworks/particles.py:173: RuntimeWarning: divide by zero encountered in divide
  Epot = -0.5 * np.sum(mass_product[:,:,np.newaxis] / np.sqrt(rij**2 + softening**2))


In [ ]:
# fixing distance to an intermidiate value
d = 20
ecc = [.5,.9,.99]
integrators = [intg.integrator_euler, intg.integrator_leapfrog, intg.integrator_hermite, intg.integrator_tsunami]

fig, ax = plt.subplots(4, 3, figsize=(10, 10))

# adaptive tstep

tstep = .001
lr = 2e-2

for k,e in enumerate(ecc):

    for i,integrator in enumerate(integrators):


        particles = ic.ic_two_body(mass1=1,mass2=1,rp=d,e=e)

        (acc_list, potential_list, positions, velocities, energy,init_energy,jerk) = evolve(particles,integrator)

        tstep = lr * np.min(np.abs(acc_list/jerk))

        ax[i,k].plot(positions[:, 0, 0], positions[:, 0, 1], label='Star 1')
        ax[i,k].plot(positions[:, 1, 0], positions[:, 1, 1], label='Star 2')

        ax[i,k].set_title(f"{integrator.__name__}  - rp = {d}")

        ax[i,k].legend()
        
fig.savefig(f"img/different_eccentricities_adaptive_lr.png")

In [11]:
(particles, time, _, _, _) = intg.integrator_tsunami(particles=particles,tstep=.001,acceleration_estimator=dyn.acceleration_direct_vectorized,softening=0.)


In [12]:
particles.pos

array([[ 0.47686519, -0.15033159,  0.        ],
       [-0.47686519,  0.15033159,  0.        ]])